In [1]:
from easy_bookmarks.integrations.linkedin import LinkedinIntegration
from dotenv import load_dotenv
import os
import polars as pl

load_dotenv()


True

In [ ]:

linkedin = LinkedinIntegration(username=os.getenv("LINKEDIN_USERNAME"), password=os.getenv("LINKEDIN_PASSWORD"))

posts = linkedin.get_bookmarked_posts()

In [4]:
df_posts = pl.DataFrame(posts)
df_posts.to_csv("df_bookmarks_800.csv", index=False)

In [65]:
df_posts = pl.read_csv("df_bookmarks_800.csv")

In [55]:
import re
import datetime


def parse_linkedin_date(parts):
    basic_map_date = {
        "m": "minutes",
        "h": "hours",
        "d": "days",
        "w": "weeks",
    }
    complex_map_date = {
        "mo": ("weeks", 4),
        "yr": ("weeks", 52),
    }

    list_match= [match for part in parts if (match := re.match(r"(\d+)\s*(\w+)", part))]

    if len(list_match) != 1:
        return None
    
    value, unit = list_match[0].groups()
    unit = unit.rstrip('s') 
    if unit in basic_map_date.keys():
        delta = datetime.timedelta(**{basic_map_date[unit]: int(value)})                #return pd.Timestamp.now() - delta
        return datetime.datetime.now().date() - delta
    else:
        complex_unit, multiplier = complex_map_date[unit]
        delta = datetime.timedelta(**{complex_unit: int(value) * multiplier})
        return datetime.datetime.now().date() - delta

## Parse date

In [66]:
df_posts = df_posts.with_columns(
    pl.col("post_date_repost_info").str.split(" • ").map_elements(parse_linkedin_date).alias("parsed_date")
).drop(["post_date_repost_info"])
df_posts.head(2)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1744\2764976344.py:1: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  df_posts = df_posts.with_columns(


author,author_catchline,author_member_id,post_summary,post_url,post_activity_id,parsed_date
str,str,i64,str,str,i64,date
"""Fred Viole""","""OVVO Financial Systems | ovvol…",24191624,"""When people ask, ""Why nonparam…","""https://www.linkedin.com/feed/…",7238576907531816960,2024-09-09
"""Leonie Monigatti""","""I do Machine Learning at Weavi…",391766404,"""If you embed an entire documen…","""https://www.linkedin.com/feed/…",7237775824395935744,2024-09-07


## Generate uuid

In [67]:
from easy_bookmarks.stores.utils import generate_uuid


df_posts.with_columns(
    pl.lit("Linkedin").alias("source"),
).with_columns(
    (pl.col("source")+pl.col("post_url")).map_elements(generate_uuid).alias("id")
).head(2)

C:\Users\Carlos\AppData\Local\Temp\ipykernel_1744\3093273126.py:6: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  ).with_columns(


author,author_catchline,author_member_id,post_summary,post_url,post_activity_id,parsed_date,source,id
str,str,i64,str,str,i64,date,str,object
"""Fred Viole""","""OVVO Financial Systems | ovvol…",24191624,"""When people ask, ""Why nonparam…","""https://www.linkedin.com/feed/…",7238576907531816960,2024-09-09,"""Linkedin""",9bfb6d07-3873-5caa-27c4-57deb3171bc2
"""Leonie Monigatti""","""I do Machine Learning at Weavi…",391766404,"""If you embed an entire documen…","""https://www.linkedin.com/feed/…",7237775824395935744,2024-09-07,"""Linkedin""",a71979ff-7571-6ae1-f015-e664cc09ced4
